In [1]:
# Global imports
import os
import sys
import pathlib
import time

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn

from torch.utils.data import DataLoader, TensorDataset
from torch.utils.data.dataset import random_split
import torch.optim as optim

from sklearn.preprocessing import MinMaxScaler

In [2]:
cwd = pathlib.Path().resolve()
src = cwd.parent
root = src.parent
sys.path.append(str(src))

In [3]:
#initialize GPU -  In case of windows use cuda instead of nps
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print("Is CUDA enabled?",torch.cuda.is_available())
print("Number of GPUs",torch.cuda.device_count())

Is CUDA enabled? True
Number of GPUs 1


- Each of the simulations used for training and validation has a total of 96 timesteps, with a spatial grid size of 64*64=4096.
- Data has already been normalized.
- Each of the training/validation simulations therefore gives 96 pairs of inputs and targets.
- Each input contains at minimum 4096 values (if only water depth is used) and at most 4096 * 4 = 16384 (if water depth, vx, vy, and topography are used).

In [4]:
from models.RNN import SimpleRNN

model = SimpleRNN(*args, **kwargs).to(device)

ModuleNotFoundError: No module named 'models.RNN'

In [6]:
# Local imports
from utils.simulation import Simulation
from train import train_and_validate, evaluate_model

In [7]:
def train_rnn(model,
              sim_amount=1,
              training_size=0.8,
              batch_size=4,
              num_epochs = 200,
              lr = 0.0005,
              criterion = nn.MSELoss(),
              optimizer = optim.AdamW,
              model_name = 'babie_first_RNN'):
    
    """
    This method trains a simple RNN. Given a single timestep consisting of water depth and topography (both 64*64), the RNN predicts a single step ahead. The best model state is
    saved following the save_path, and also returned by the method.
    
    Description of arguments:
    - model: the model to be trained, should be an instance of the class SimpleRNN;
    - sim_amount (int): number of simulations of which the data is loaded and used for training, with a maximum of 400;
    - training_size (float): fraction of data to use for training (validation uses the fraction 1 - training_size);
    - batch_size (int): batch size used during training (you can modify this based on your requirements);
    - num_epochs (int): number of epochs used during training;
    - lr (float): learning rate used during training;
    - criterion: Loss function, default nn.MSELoss()
    - optimizer: optimizer used for training, default optim.AdamW
    - model_name (string): the best model state will be saved in ../results/trained_models/ under this name

    returns: model, train_losses, val_losses, best_val_loss, time
    """
    # load simulations to be used for training
    sims = Simulation.load_simulations(str(root)+"/data/processed_data/normalized_training_data", sim_amount=sim_amount, number_grids=64)

    n_timesteps = 96
    grid_size = 64
    channels = 2   # water depth and topography

    # reformat the data
    X = np.zeros((n_timesteps*len(sims), grid_size, grid_size, channels))   # timestep * grid_x * grid_y * channels
    Y = np.zeros(X[:,:,:,0].shape)   # timestep * grid_x * grid_y

    for i in range(len(sims)):   # number of simulations loaded in for training/validation

        sim = sims[i]            # get simulation
        topography = sim.topography

        for t_i in range(n_timesteps):    # number of timesteps

            wd, vx, vy = sim.return_timestep(t_i)
            X[t_i+i*n_timesteps, :, :, 0] = wd
            X[t_i+i*n_timesteps, :, :, 1] = topography

            wd, vx, vy = sim.return_timestep(t_i+1)
            Y[t_i+i*n_timesteps, :, :] = wd

    # split the data into training and validation
    id_training = int(training_size * len(X))

    X_tra = X[:id_training, :]
    Y_tra = Y[:id_training, :]

    X_val = X[id_training:, :]
    Y_val = Y[id_training:, :]

    print("X_tra.shape: ", X_tra.shape)
    print("Y_tra.shape: ", Y_tra.shape)
    print("X_val.shape: ", X_val.shape)
    print("Y_val.shape: ", Y_val.shape)

    #create datasets and data loaders
    train_dataset = TensorDataset(torch.tensor(X_tra, dtype=torch.float32), torch.tensor(Y_tra, dtype=torch.float32))
    val_dataset = TensorDataset(torch.tensor(X_val, dtype=torch.float32), torch.tensor(Y_val, dtype=torch.float32))

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    # defining the optimizer
    optimizer = optimizer(model.parameters(), lr=lr)

    # defining the save path
    save_path = "../results/trained_models/" + model_name

    # training
    train_losses, val_losses, best_val_loss, time = train_and_validate(model, train_loader, val_loader, criterion, optimizer, num_epochs, device, save_path)

    # Load the best model
    model.load_state_dict(torch.load(save_path))

    return model, train_losses, val_losses, best_val_loss, time

In [8]:
train_rnn(model,
          sim_amount=1,
          training_size=0.8,
          batch_size=4,
          num_epochs = 200,
          lr = 0.0005,
          criterion = nn.MSELoss(),
          optimizer = optim.AdamW,
          model_name = 'babies_first_RNN')

NameError: name 'model' is not defined

X_tra.shape:  (76, 64, 64, 2)
Y_tra.shape:  (76, 64, 64)
X_val.shape:  (20, 64, 64, 2)
Y_val.shape:  (20, 64, 64)
